## Import Libraries

In [ ]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer, util
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import torch
import os
from glob import glob
import torch
import numpy as np
from zero_dce import (
    Trainer, plot_result
)


In [3]:
print(torch.cuda.is_available())
# print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")
model = SentenceTransformer('clip-ViT-L-14')

True


In [4]:
def enhance_images_with_zero_dce(full_image_paths, model_weights_path):
    trainer = Trainer()
    trainer.build_model(pretrain_weights=model_weights_path)

    enhanced_paths = []

    for path in full_image_paths:
        path = os.path.normpath(path)
        image, enhanced = trainer.infer_gpu(path, image_resize_factor=1)

        alpha = 0.4
        enhanced_toned = alpha * enhanced + (1 - alpha) * np.asarray(image) / 255.0
        enhanced_toned = np.clip(enhanced_toned, 0, 1)

        enhanced_path = path.replace("tpc-imgs", "enhanced-imgs")
        os.makedirs(os.path.dirname(enhanced_path), exist_ok=True)
        Image.fromarray((enhanced_toned * 255).astype(np.uint8)).save(enhanced_path)

        enhanced_paths.append(enhanced_path)

    return enhanced_paths

## Image Preprocessing

In [6]:
# Cell 2: Load prompts and unique labels from JSON
def load_dataset(json_path):
    with open(json_path) as f:
        data = json.load(f)
    # image_paths = ['/content/' + entry['image_path'] for entry in data]
    image_paths = ['C:\College\Semester 4\Research\\' + entry['image_path'] for entry in data]
    descriptions = [entry['description'] for entry in data]
    slangs = [entry['slang'] for entry in data]
    # slangs = [entry['label'] for entry in data]

    label_to_desc = {}
    for desc, slang in zip(descriptions, slangs):
        label_to_desc[slang] = desc

    unique_slangs = list(label_to_desc.keys())
    unique_descriptions = list(label_to_desc.values())

    return image_paths, slangs, unique_slangs, unique_descriptions

In [7]:
# Cell 3: Encode descriptions into text embeddings
def encode_text_prompts(model, descriptions):
    text_embeddings = model.encode(descriptions, convert_to_tensor=True, normalize_embeddings=True)
    return text_embeddings

## Inference

In [8]:
# Cell 4: Run inference on images, returning predicted slang labels
def run_inference_batch(image_paths, model, text_embeddings, unique_slangs, batch_size=5):
    predictions = []
    imgs_batch = []

    for i, img_path in enumerate(image_paths):
        img = Image.open(img_path).convert('RGB')
        imgs_batch.append(img)

        if (i + 1) % batch_size == 0 or (i + 1) == len(image_paths):
            img_embs = model.encode(imgs_batch, convert_to_tensor=True, normalize_embeddings=True)
            cos_sim = util.cos_sim(img_embs, text_embeddings)
            pred_idxs = cos_sim.argmax(dim=1).tolist()
            for idx in pred_idxs:
                predictions.append(unique_slangs[idx])
            imgs_batch = []
    return predictions

## Evaluation

In [9]:
# Cell 5: Evaluate predictions with accuracy, precision, recall, and confusion matrix
def evaluate(true_labels, pred_labels, unique_slangs):
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, labels=unique_slangs, average=None, zero_division=0)
    recall = recall_score(true_labels, pred_labels, labels=unique_slangs, average=None, zero_division=0)
    cm = confusion_matrix(true_labels, pred_labels, labels=unique_slangs)

    print(f"Accuracy: {accuracy*100:.2f}%\n")

    for i, label in enumerate(unique_slangs):
        print(f"Label: {label}")
        print(f"  Precision: {precision[i]:.3f}")
        print(f"  Recall:    {recall[i]:.3f}\n")

    print("Confusion Matrix:")
    print(cm)


In [17]:
%cd /College/Semester 4/Research
%pwd
# Cell 6: Main script - load data, model, run inference, evaluate
if __name__ == "__main__":
    # json_path = "/content/metadata_balanced.json"
    json_path = "metadata_unbalanced.json"  # or "metadata_balanced.json"
    base_dir = "C:/College/Semester 4/Research"
    model_weights_path = r"C:/College/Semester 4/Research/Zero-DCE/pretrained-models/model200_dark_faces.pth"

    image_paths, true_slangs, unique_slangs, unique_descriptions = load_dataset(json_path)
    image_paths = [os.path.join(base_dir, p) for p in image_paths]

    if "unbalanced" in json_path.lower():
        print("[INFO] Unbalanced dataset detected — applying Zero-DCE enhancement...")
        image_paths = enhance_images_with_zero_dce(
            full_image_paths=image_paths,
            model_weights_path=model_weights_path
        )
        
    text_emb = encode_text_prompts(model, unique_descriptions)

    print("[INFO] Running inference...")
    predicted_slangs = run_inference_batch(image_paths, model, text_emb, unique_slangs)

    print("[INFO] Evaluating...")
    evaluate(true_slangs, predicted_slangs, unique_slangs)

c:\College\Semester 4\Research\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


C:\College\Semester 4\Research
[INFO] Unbalanced dataset detected — applying Zero-DCE enhancement...
[INFO] Running inference...
[INFO] Evaluating...
Accuracy: 75.00%

Label: Virginia creeper
  Precision: 0.933
  Recall:    0.718

Label: Boxelder
  Precision: 0.500
  Recall:    0.846

Confusion Matrix:
[[28 11]
 [ 2 11]]
